# 🔰 Notebook 3 — Gateway + On‑prem Deploy (Phase 1, end of Module I)

> **Goal:** run the **Gateway (8080)** → **tasks‑service (8081)** stack, validate **routing** via `/api/tasks/**`,
> use **X‑API‑KEY** (optional), verify **observability**, and consolidate **on‑prem deployment**.
>
> **Didactic order:** start with **STS (dev)** to understand/debug; then **Docker Compose** (📌 **recommended** for stable, multi‑service runs).  
> **No** Eureka/Config/Kubernetes in this module.

---

## 0) API Gateway — what it is, why it’s used, AWS/Azure mapping, and how it differs from a Load Balancer

### 0.1 What is an API Gateway (cloud context)
An **API Gateway** is the **single north–south entry point** for your system’s APIs. It:
- **Receives** client requests (web, mobile, integrations).
- **Applies policies**: authentication/authorization, API keys, **rate limiting**/throttling, CORS.
- **Transforms** requests/responses: rewrite paths/headers, normalize payloads.
- **Routes** each call to the correct service and can **version** APIs (v1, v2…).
- **Observes**: health/metrics, per‑route logging, dashboards.
- **Decouples** clients from internal services (names/ports/topology).

> **In Phase 1** we implement this role with **Spring Cloud Gateway (SCG)** — lightweight, didactic, and ideal for local/on‑prem execution.

### 0.2 Why use an API Gateway now
- **Single entry** → clients call `http://host:8080/api/**` instead of discovering each service.
- **Consistency** → common headers, CORS, error shapes, and versioning in one place.
- **Minimal security** → in this MVP, **X‑API‑KEY** (optional); later IAM/JWT/TLS/rate‑limit.
- **Controlled evolution** → swap/replicate services without breaking clients.
- **Observability** → health/logs per route; later metrics/tracing.

### 0.3 What we implement **now** with Spring Cloud Gateway
- **Route:** `/api/tasks/**` → `tasks-service:8081`  
- **API key (optional):** if env var `API_KEY` exists in the gateway process, header `X-API-KEY` is **required** (missing/wrong → **401**).  
  Without `API_KEY`, the gateway runs in **open mode** (dev/demo).  
- **Observability:** `GET /actuator/health` (UP/DOWN).

🔗 **Twelve‑Factor touchpoints:** **#7 Port Binding**, **#3 Config**, **#5 Build/Release/Run**, **#6 Processes (stateless)**, **#10 Dev/Prod parity**, **#11 Logs**, **#9 Disposability**.

### 0.4 Mapping this to **AWS** (forward‑looking; not implemented here)
- **Managed equivalent:** **AWS API Gateway** (REST/HTTP API) as the **public entry**.  
- **Typical backends:** **ALB** in front of **ECS/EKS** (containers) or direct to **Lambda/ALB**.  
- **Policies:** API Keys, **JWT/OAuth2** with **Cognito**, throttling/quotas, WAF.  
- **Observability:** **CloudWatch** logs/metrics, X‑Ray for tracing.  
- **Analogy to our MVP:** what SCG does (route + API key) maps to API Gateway policies (keys/authorizers) targeting a backend (ALB/ECS/EKS).

### 0.5 Mapping this to **Azure** (forward‑looking; not implemented here)
- **Managed equivalent:** **Azure API Management (APIM)** as public entry.  
- **Typical backends:** **App Service**, **AKS**, **Container Apps**, **Application Gateway**.  
- **Policies:** subscription keys, **JWT/OAuth2** via **Microsoft Entra ID**, rate‑limit, rewrite, cache, transformations.  
- **Observability:** **Azure Monitor/Log Analytics**, optional Developer Portal.  
- **Analogy to our MVP:** APIM policy (subscription key/JWT) → backend in AKS/Apps; same conceptual shape as SCG → `tasks-service`.

### 0.6 **Load Balancer** — what it is and how it differs from the Gateway
A **Load Balancer (LB)** distributes traffic across **instances** of the same service. It focuses on:
- **Balancing** (round‑robin, least‑connections…), **health checks**, **failover**.
- Terminating **TLS** and operating at L4 (TCP) or L7 (HTTP), but it’s **not** an API governance layer.
- It **does not** manage API versioning, rich API keys, rate‑limit policies, or payload transformations.

**Examples:**
- **AWS**: **ALB/NLB** (Application/Network LB).  
- **Azure**: **Application Gateway** (L7), **Azure Load Balancer** (L4).

> **Common pattern:** put an **API Gateway** in front and an **LB** behind (gateway → LB → services/containers).  
> In our MVP, the **SCG gateway** is the entry point; **balancing** is not needed yet.

### 0.7 Summary table (Gateway vs Load Balancer)

| Aspect | **API Gateway** (SCG / AWS API GW / Azure APIM) | **Load Balancer** (ALB/NLB / App GW/LB) |
|---|---|---|
| Traffic | **North–south** (client → system) | North–south (distribute to instances) |
| Layer | L7 (API‑centric) | L4/L7 (distribution‑centric) |
| API policies | **Rich** (keys, JWT/OAuth2, rate‑limit, CORS) | **Limited**/none |
| Transformations | **Yes** (headers/path/payload) | Limited |
| Versioning | **Yes** (v1/v2, routes) | No |
| API observability | **Yes** (per route/API) | Basic (per target) |
| Dev portal | (APIM/API GW) **Yes** | No |
| Health checks | Yes (edge; API focus) | **Yes** (instances) |
| Typical use | Edge, API governance | Distribute to replicas |

---

## 1) Code and phase

```bash
git clone https://github.com/smartlearningci/cloud_java
cd cloud_java
git checkout phase-1
```

**Included:**  
- `tasks-service` (Spring Boot; **8081**; H2; Actuator)  
- `gateway` (Spring Cloud Gateway; **8080**; route `/api/tasks/**`)  
- **X‑API‑KEY** optional (enforced **only** if `API_KEY` is set in the gateway environment)

🔗 **Twelve‑Factor #1 Codebase** — phase tags = stable states; the class stays aligned by checkpoints.

---

## 2) **First:** run with **STS** (development)

> Best for **debugging**, seeing each service’s logs, and understanding the mechanics before automating.

### 2.1 Start `tasks-service` (8081)
- STS → **File > Import > Maven > Existing Maven Projects** → select the `tasks-service` project  
- Open the main class (e.g., `…TasksServiceApplication`)  
- **Run As > Spring Boot App**  
- Verify: the app listens on **8081**; `GET http://localhost:8081/actuator/health` → `{"status":"UP"}`

### 2.2 Start `gateway` (8080)
- Import the `gateway` project  
- (Optional) to require API key: set **`API_KEY`** in the gateway **Run Configuration**  
- **Run As > Spring Boot App**  
- Verify: gateway listens on **8080**; `GET http://localhost:8080/actuator/health` → `{"status":"UP"}`

### 2.3 Test **via the Gateway** (Postman recommended; `curl` optional)
- **Postman Environment:**
  - `baseUrl = http://localhost:8080/api`
  - `apiKey = <your_value>` (only if `API_KEY` is active)
- **Requests:**
  1) **GET** `{{baseUrl}}/tasks` → `200 OK`  
     - If `API_KEY` active: header `X-API-KEY: {{apiKey}}` (otherwise → **401**)
  2) **POST** `{{baseUrl}}/tasks` (JSON):
     ```json
     {
       "title": "Task via Gateway",
       "description": "Unified Notebook - Phase 1",
       "projectId": "GW",
       "assignee": "Team"
     }
     ```
  3) **PATCH** `{{baseUrl}}/tasks/{{taskId}}/status` (JSON `{ "status": "DOING" }`)
  4) **GET** `{{baseUrl}}/tasks/{{taskId}}`

**`curl` equivalents:**
```bash
# without API_KEY active
curl -s http://localhost:8080/api/tasks | jq

# with API_KEY=supersecret
curl -s -H "X-API-KEY: supersecret" http://localhost:8080/api/tasks | jq
curl -s -H "X-API-KEY: supersecret" -H "Content-Type: application/json"   -d '{"title":"Task via Gateway","description":"Unified Notebook - Phase 1","projectId":"GW","assignee":"Team"}'   http://localhost:8080/api/tasks | jq
```

🔗 **Twelve‑Factor:** **#7 Port Binding**, **#11 Logs**, **#3 Config**, **#5 Build/Release/Run** — single entry `/api/**`, separate logs, behavior changes via **env var** (no rebuild).

> ⚠️ **Good practice:** processes started from the IDE should **not** be exposed directly to the public Internet.

---

## 3) **Then:** run with **Docker Compose** (📌 **recommended**)

> To run the stack **reliably**, reproducibly, and closer to a real multi‑service environment.

### 3.1 (Optional) Enable `X-API-KEY`
Choose **ONE** method:

- **Environment variable (session):**
  ```bash
  export API_KEY=supersecret
  ```
- **`.env` file** at the project root:
  ```
  API_KEY=supersecret
  ```

> Without `API_KEY`, the gateway **does not** require the header (open dev/demo mode).

### 3.2 Bring up the stack
```bash
./run_compose.sh
# or:
# docker compose up --build
```

### 3.3 Validate health
- Gateway: `http://localhost:8080/actuator/health` → `{"status":"UP"}`
- Backend: `http://localhost:8081/actuator/health` → `{"status":"UP"}`

### 3.4 Useful logs
```bash
docker compose logs -f gateway
docker compose logs -f tasks-service
```

🔗 **Twelve‑Factor:** **#5 Build/Release/Run**, **#7 Port Binding**, **#9 Disposability**, **#10 Dev/Prod parity**, **#3 Config**.

---

## 4) Observability & quick diagnosis

- **Health:**
  - Gateway → `GET /actuator/health`
  - Backend → `GET /actuator/health`
- **Expected errors:**
  - **401 (gateway)**: `API_KEY` active and `X-API-KEY` missing/wrong
  - **404 (backend via gateway)**: non‑existent id/wrong route
  - **409/422 (backend)**: invalid state transition
  - **502/503 (gateway)**: backend unavailable (check `:8081/actuator/health` and logs)
- **Ports busy (8080/8081)**: stop old instances/containers or change ports in the local profile.

🔗 **Twelve‑Factor:** **#9 Disposability** (simple health reduces MTTR), **#10 Dev/Prod parity** (predictable errors across environments).

---

## 5) Safe exposure (MVP on‑prem)

> **Heads‑up:** Phase 1 has **no TLS** and **no user auth**.  
> If you must expose it, do it **briefly**, with **`API_KEY` enabled**, and basic mitigations:

- Use a **non‑default** key (rotate if shared).
- **Restrict source IPs** (firewall/security group).
- Prefer **VPN/tunnel** (e.g., WireGuard/SSH) over opening ports.
- (Optional) Place a **reverse proxy** (Nginx/Caddy) in front of the gateway to terminate **TLS**.
- Watch **logs** while exposed.
- Remember: **H2 is in‑memory** — data resets on restart (persistence arrives in Module III).

---

## 6) Tools used (and why)

- **Spring Boot**: rapid start, REST, Actuator (health).
- **Spring Cloud Gateway**: single entry, simple policy (X‑API‑KEY), clear routing.
- **Git**: phase tags → didactic checkpoints.
- **STS**: great DX for run/debug locally.
- **Docker Compose**: predictable multi‑service on one host (📌 **recommended** to run the stack).
- **Postman**: shareable requests (alternative: `curl`).

---

## 7) Quick troubleshooting

- **8080/8081 busy** → stop old instances or change ports.
- **API_KEY not applied** → ensure `.env` is correct or `export API_KEY=...` **before** starting; in STS, set it in **Run Configuration**.
- **Gateway 502/503** → backend not UP or wrong route/config; check logs.
- **Data “disappears”** → expected with **H2 in‑memory** (restart clears data).

---

## ✅ Checkpoints (no quizzes)

- [ ] I started via **STS**: `tasks-service (8081)` and `gateway (8080)` are **UP**.  
- [ ] I exercised tests **via the gateway** (`/api/tasks/**`) and saw separate logs.  
- [ ] I brought the stack up with **Docker Compose** (📌 **recommended**) and validated **/actuator/health** on both services.  
- [ ] With `API_KEY` active, I observed **401** without header and **200/201** with the correct `X-API-KEY`.  
- [ ] I understand **why we use a gateway**, how it maps to **AWS/Azure**, and key **differences vs a Load Balancer**.  
- [ ] I understand MVP exposure risks and basic on‑prem mitigations.

---
